<a href="https://colab.research.google.com/github/Radperia/SIGNATE_student_2020/blob/master/StackingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 / 60 / 60 "h)"}'

0.00581782days (0.139628h)


In [4]:
!git clone https://github.com/microsoft/LightGBM
!cd LightGBM && mkdir build && cd build && cmake .. && make -j"$(nproc)"
!cd LightGBM/python-package && python setup.py install --precompile

Cloning into 'LightGBM'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 18883 (delta 7), reused 9 (delta 3), pack-reused 18855
Receiving objects: 100% (18883/18883), 12.71 MiB | 22.60 MiB/s, done.
Resolving deltas: 100% (13794/13794), done.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found Op

In [13]:
import numpy as np
import pandas as pd
import time
import re
import string
import gc
import os
import psutil
import collections
import lightgbm as lgb
from contextlib import contextmanager
from collections import defaultdict
from google.colab import files
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
from keras.preprocessing.text import Tokenizer
from sklearn import metrics
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [6]:
train_df = pd.read_csv('./drive/My Drive/SIGNATE/train.csv').fillna(' ')
test_df  = pd.read_csv('./drive/My Drive/SIGNATE/test.csv').fillna(' ')
sample = pd.read_csv('./drive/My Drive/SIGNATE/submit_sample.csv', header=None)

train_text = train_df['description']
test_text = test_df['description']
all_text = pd.concat([train_text, test_text])

sentences = train_df['description'].tolist() + test_df['description'].tolist()
sentences[::500]  # 500個飛ばしに確認

['Executes and writes portions of testing plans, protocols, and documentation for assigned portion of application; identifies and debugs issues with code and suggests changes or improvements.',
 'Knowledge and experience of complex software design for distributed systems in embedded networking/telecommunications projects.',
 'Analyze client business processes of storing and managing data.',
 'Defining tasks, timeline and required resources to deliver on the mechanical design portion of a project brief, with strong accountability for successful completion of agreed upon deliverables.',
 'Familiar with sensors, transducers, physiologic modeling, and transfer functions',
 'Manage, maintain, refresh and upgrade environmental components (including patching, etc.)',
 'Conduct research on cutting-edge techniques and tools in machine learning/deep learning/artificial intelligence',
 'Assist in planning and development of visualizations and analytic aggregations to best portray discovered insig

In [7]:
tokenizer = Tokenizer(num_words=1000)  # 出現頻度上位{num_words}だけを用いる
tokenizer.fit_on_texts(sentences)
print(list(tokenizer.word_index)[:10] ) # 学習された辞書（出現頻度順）

['and', 'to', 'the', 'of', 'with', 'data', 'in', 'for', 'a', 'business']


In [8]:
# 学習・検証に分けて，ベクトルを生成。加えて，正解ラベルも作成
train_X, test_X = np.split(tokenizer.texts_to_matrix(sentences, mode='binary'),
                           [len(train_df)], axis=0)

train_y = train_df['jobflag'].values - 1  # maps {1, 2, 3 ,4} -> {0, 1, 2, 3}
train_X.shape, train_y.shape, test_X.shape

((2931, 1000), (2931,), (1743, 1000))

In [9]:
rfc_param = {
    'rfc__learning_rate': [0.001],
    'rfc__n_estimators': [100, 500],
    'rfc__max_depth': [10, 50, 100],
    'rfc__bootstrap': ['True'],
    'rfc__passthrough': ['True'],
    'rfc__class_weight': ['balanced']
}

rfc = RandomForestClassifier()
rfc_cv = GridSearchCV(estimator=rfc, param_grid=rfc_param, cv=5, verbose=3, n_jobs=-1, scoring='%s_weighted'%'f1')

In [10]:
grad_param = {
    'grad__learning_rate': [0.001],
    'grad__n_estimators': [100, 500],
    'grad__max_depth': [10, 50, 100]
}
grad = GradientBoostingClassifier()
grad_cv = GridSearchCV(estimator=grad, param_grid=grad_param, cv=5, verbose=3, n_jobs=-1, scoring='%s_weighted'%'f1')

In [11]:
mlp_param = {
    'mlp__hidden_layer_sizes': [100, 200, 300],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__solver': ['adam'],
    'mlp__batch_size': ['auto'],
    'mlp__learning_rate': [0.001, 0.0001],
}
mlp = MLPClassifier()
mlp_cv = GridSearchCV(estimator=mlp, param_grid=mlp_param, cv=5, verbose=3, n_jobs=-1, scoring='%s_weighted'%'f1')

一気にやると上手く行かないから、個別に最適パラメータを求めて、最後にスタッキングする

In [35]:
from mlxtend.classifier import StackingClassifier

clf1 = RandomForestClassifier(random_state=1)
clf2 = GradientBoostingClassifier()
clf3 = MLPClassifier(solver='adam', batch_size='auto')
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=lr, verbose=3)

params = {
        'randomforestclassifier__n_estimators': [100, 500],
        'randomforestclassifier__max_depth': [10, 50, 100],
        'randomforestclassifier__class_weight': ['balanced', 'balanced_subsample'],
        'gradientboostingclassifier__learning_rate': [0.001, 0.01],
        'gradientboostingclassifier__n_estimators': [100, 500],
        'gradientboostingclassifier__max_depth': [3, 10, 50],
        'mlpclassifier__hidden_layer_sizes': [100, 200, 300],
        'mlpclassifier__activation': ['relu', 'tanh'],
}

grid = GridSearchCV(estimator=sclf, param_grid=params, cv=5, refit=True)
#print(sclf.get_params().keys())
grid.fit(train_X, train_y)

Fitting 3 classifiers...
Fitting classifier1: randomforestclassifier (1/3)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=1,
                       warm_start=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


Fitting classifier2: gradientboostingclassifier (2/3)
GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.001, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=1,
                           warm_start=False)
      Iter       Train Loss   Remaining Time 
         1        2948.2195           17.82s
         2        2947.0421           18.54s
         3        2945.8683           18.13s
         4        2944.6980           17.82s
         5        294

KeyboardInterrupt: ignored

In [12]:
'''
estimators = [
              ('rfc', rfc),
              ('grad', grad),
              ('mlp', mlp)
]

clf_stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=1000))
clf_stack.fit(train_X, train_y)
'''

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


StackingClassifier(cv=None,
                   estimators=[('rfc',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                    